In [ ]:
# 多种联合分析
python $HOME/softwares/ldsc/ldsc.py \
--h2 ${dir_GWAS}/${trait}.sumstats.gz \
--ref-ld-chr ${dir_ldsc_annot}/${name_annot}/${name_annot}.,${dir_ldsc_annot}/${m6A_annot}/${m6A_annot}.,${dir_ldsc_annot}/${eQTL_annot}/${eQTL_annot}.,${dir_ldsc_annot}/${sQTL_annot}/${sQTL_annot}. \
--frqfile-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_frq/1000G.EUR.QC. \
--w-ld-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_weights_hm3_no_MHC/weights.hm3_noMHC. \
--overlap-annot --print-cov --print-coefficients --print-delete-vals \
--out ${dir_out}/${trait}_${m6A_annot}_eQTL_sQTL_PIP_${name_annot}

## Joint with eQTL PIP
python $HOME/softwares/ldsc/ldsc.py \
--h2 ${dir_GWAS}/${trait}.sumstats.gz \
--ref-ld-chr ${dir_ldsc_annot}/${name_annot}/${name_annot}.,${dir_ldsc_annot}/${m6A_annot}/${m6A_annot}.,${dir_ldsc_annot}/${eQTL_annot}/${eQTL_annot}. \
--frqfile-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_frq/1000G.EUR.QC. \
--w-ld-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_weights_hm3_no_MHC/weights.hm3_noMHC. \
--overlap-annot --print-cov --print-coefficients --print-delete-vals \
--out ${dir_out}/${trait}_${m6A_annot}_eQTL_PIP_${name_annot}

python $HOME/softwares/ldsc/ldsc.py \
--h2 ${dir_GWAS}/${trait}.sumstats.gz \
--ref-ld-chr ${dir_ldsc_annot}/${name_annot}/${name_annot}.,${dir_ldsc_annot}/${m6A_annot}/${m6A_annot}. \
--frqfile-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_frq/1000G.EUR.QC. \
--w-ld-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_weights_hm3_no_MHC/weights.hm3_noMHC. \
--overlap-annot --print-cov --print-coefficients --print-delete-vals \
--out ${dir_out}/${trait}_${m6A_annot}_${name_annot}

In [2]:
import pandas as pd
import os
from subprocess import call

In [13]:
# 处理gwas数据
def get_gwas_sumstats(gwas_type):
    gwas_file = f"/mnt/hpc/home/xuxinran/DirectSeq/4_gwas/8diseases/credible_set/{gwas_type}_credible_set.csv"
    meta_file = f"/mnt/hpc/home/xuxinran/DirectSeq/4_gwas/8diseases/meta/{gwas_type}.csv"
    df = pd.read_csv(gwas_file, usecols = ['chr','bp','rsid',"maf",'ea',"nea",'beta','se','p','zscore','meta_id'])
    meta_df = pd.read_csv(meta_file, usecols = ['meta_id',"sample_size","n_case","n_control","popu"])
    df = pd.merge(df, meta_df, on='meta_id', how='outer')
    # df = df[df['popu']=='EAS'] # 筛选EAS
    df = df[df['popu'].isin(['UKBB', 'EUR'])] # 筛选EUR
    df['chr'] = df['chr'].astype(int)
    df = df.dropna()
    if df.empty:
        print(f"{gwas_type} is empty")
        return
    dfs = df.groupby('meta_id')
    for meta_id, group in dfs:
        group = group.reset_index()
        popu = group.at[0,"popu"]
        sample_size = int(group.at[0,"sample_size"])
        n_case = int(group.at[0,"n_case"])
        n_control = int(group.at[0,"n_control"])
        res_df = group[['rsid','chr','bp','maf','ea','nea','beta','se','p','zscore']]
        res_df.to_csv(f"{gwas_type}_{popu}_{meta_id}.txt", sep='\t', index=False)
        print(f'python /mnt/hpc/home/xuxinran/GWAS/ldsc/munge_sumstats.py --sumstats {gwas_type}_{popu}_{meta_id}.txt --merge-alleles /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/w_hm3.snplist --out {gwas_type}_{popu}_{meta_id} --a1-inc --a1 ea --a2 nea --p P --frq maf --snp rsid --N {sample_size} --N-cas {n_case} --N-con {n_control}')

os.chdir("/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats")
# gwas_type_l = ["Autoimmune","Cardiovascular","Digestive","Endocrine","Musculoskeletal","Neoplasms","Nervous","Skin"]
gwas_type_l = ["Cardiovascular"]
for i in gwas_type_l:
    get_gwas_sumstats(i)

python /mnt/hpc/home/xuxinran/GWAS/ldsc/munge_sumstats.py --sumstats Cardiovascular_EUR_CA448.txt --merge-alleles /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/w_hm3.snplist --out Cardiovascular_EUR_CA448 --a1-inc --a1 ea --a2 nea --p P --frq maf --snp rsid --N 547261 --N-cas 122733 --N-con 424528
python /mnt/hpc/home/xuxinran/GWAS/ldsc/munge_sumstats.py --sumstats Cardiovascular_EUR_DB714.txt --merge-alleles /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/w_hm3.snplist --out Cardiovascular_EUR_DB714 --a1-inc --a1 ea --a2 nea --p P --frq maf --snp rsid --N 1345 --N-cas 142 --N-con 1203
python /mnt/hpc/home/xuxinran/GWAS/ldsc/munge_sumstats.py --sumstats Cardiovascular_EUR_DB716.txt --merge-alleles /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/w_hm3.snplist --out Cardiovascular_EUR_DB716 --a1-inc --a1 ea --a2 nea --p P --frq maf --snp rsid --N 1345 --N-cas 142 --N-con 1203
python /mnt/hpc/home/xuxinran/GWAS/ldsc/munge_sumstats.py --sumstats Cardiovascular_EUR

In [4]:
# 处理qtl数据
def get_Qtl_bed(qtltype,qtl_file):
    if qtltype == "stQTL":
        df = pd.read_csv(qtl_file,usecols = ['chrom','snp_pos_1base','rsID','pvalue'])
        df = df[df['pvalue']<0.05]
    else:
        df = pd.read_csv(qtl_file,usecols = ['chrom','snp_pos_1base','rsID','BayesFactor'])
        df = df[df['BayesFactor']>3]
    df = df[['chrom','snp_pos_1base','rsID']]
    # df['chrom'] = df['chrom'].str.replace('chr', '')
    df = df.drop_duplicates()
    df['s'] = df['snp_pos_1base']-1
    df['e'] = df['snp_pos_1base']+1
    df = df[['chrom','s','e']]
    df.to_csv(f"{qtltype}.bed", sep='\t', index=False, header=False)
    # call(f'sort -k1,1 -k2,2n {qtltype}.bed > {qtltype}_sort.bed',shell=True)
    for i in list(range(1,23)):
        # print(f'python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file {qtltype}.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC.{i}.bim --annot-file EAS/{qtltype}.{i}.annot.gz')
        print(f'python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file {qtltype}.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/1000G.EUR.QC.{i}.bim --annot-file EUR/{qtltype}.{i}.annot.gz')

qtl_dict = {
        'inosineQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/Iqtl/nano_merge_I_summary.csv',
        'puQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/puqtl/nano_merge_promoter_summary.csv',
        'm6AQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/m6Aqtl/nano_merge_m6A_summary.csv',
        'pseUQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/pseUqtl/nano_merge_pseU_summary.csv',
        'm5CQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/m5Cqtl/nano_merge_m5C_summary.csv',
        'stQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/stqtl/nano_merge_stability_summary.csv',
        '3aQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/3aqtl/nano_merge_APA_summary.csv',
        'irQTL':'/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/irqtl/nano_merge_isoform_summary.csv'
    }
for qtltype,qtl_file in qtl_dict.items():
    os.chdir(f"/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot")
    get_Qtl_bed(qtltype,qtl_file)



python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file inosineQTL.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/1000G.EUR.QC.1.bim --annot-file EUR/inosineQTL.1.annot.gz
python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file inosineQTL.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/1000G.EUR.QC.2.bim --annot-file EUR/inosineQTL.2.annot.gz
python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file inosineQTL.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/1000G.EUR.QC.3.bim --annot-file EUR/inosineQTL.3.annot.gz
python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file inosineQTL.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/1000G.EUR.QC.4.bim --annot-file EUR/inosineQTL.4.annot.gz
python /mnt/hpc/home/xuxinran/GWAS/ldsc/make_annot.py --bed-file inosineQTL.bed --bimfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_P

In [ ]:
## annot制作l2
import subprocess

chrom_dict = {}
qtl_list = ['inosineQTL','puQTL','m6AQTL','pseUQTL','m5CQTL','stQTL','3aQTL','irQTL']
for qtltype in qtl_list:
  for i in list(range(1,23)):
      l = []
      try:
        result = subprocess.run(f'zgrep -q -e 1 /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/{qtltype}.{i}.annot.gz', shell=True, check=True, capture_output=True)
        if result:
          l.append(i)
          # print(f'python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --l2 --bfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC.{i} --print-snps /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/listHM3.txt --ld-wind-cm 1 --annot /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/{qtltype}.{i}.annot.gz --thin-annot --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/{qtltype}.{i}')
          print(f'python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --l2 --bfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC.{i} --print-snps /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/listHM3.txt --ld-wind-cm 1 --annot /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/{qtltype}.{i}.annot.gz --thin-annot --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/{qtltype}.{i}')
      except subprocess.CalledProcessError:
        pass
  chrom_dict[qtltype] = l

In [4]:
qtl_list = ['inosineQTL','puQTL','m6AQTL','pseUQTL','m5CQTL','stQTL','3aQTL','irQTL']
for qtltype in qtl_list:
    for i in list(range(1,23)):
        print(f'python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --l2 --bfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC.{i} --print-snps /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/listHM3.txt --ld-wind-cm 1 --annot /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/EAS/{qtltype}.{i}.annot.gz --thin-annot --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/EAS/{qtltype}.{i}')

python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --l2 --bfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC.1 --print-snps /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/listHM3.txt --ld-wind-cm 1 --annot /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/EAS/inosineQTL.1.annot.gz --thin-annot --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/EAS/inosineQTL.1
python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --l2 --bfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC.2 --print-snps /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr/listHM3.txt --ld-wind-cm 1 --annot /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/EAS/inosineQTL.2.annot.gz --thin-annot --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/EAS/inosineQTL.2
python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --l2 --bfile /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phas

In [ ]:
python $HOME/softwares/ldsc/ldsc.py \
--l2 \
--bfile ${dir_LDSC}/LDSCORE/1000G_EUR_Phase3_plink/1000G.EUR.QC.${chrom} \
--print-snps ${dir_LDSC}/LDSCORE/listHM3.txt \
--ld-wind-cm 1 \
--annot ${dir_LDSC}/annot/ldscores/${prefix_annot}/${prefix_annot}.${chrom}.annot.gz \
--thin-annot --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/${prefix_annot}.${chrom}

In [15]:
gwas_files = []
for filename in os.listdir("/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats"):
    if filename.endswith(".sumstats.gz"):
        gwas_files.append(filename)

qtl_list = ['inosineQTL','puQTL','m6AQTL','pseUQTL','m5CQTL','stQTL','3aQTL','irQTL']

In [ ]:
python $HOME/softwares/ldsc/ldsc.py \
--h2 ${dir_GWAS}/${trait}.sumstats.gz \
--ref-ld-chr ${dir_ldsc_annot}/${name_annot}/${name_annot}.,${dir_ldsc_annot}/${m6A_annot}/${m6A_annot}. \
--frqfile-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_frq/1000G.EUR.QC. \
--w-ld-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_weights_hm3_no_MHC/weights.hm3_noMHC. \
--overlap-annot --print-cov --print-coefficients --print-delete-vals \
--out ${dir_out}/${trait}_${m6A_annot}_${name_annot}

In [18]:
for i in gwas_files:
    gwas_id = i.replace(".sumstats.gz","")
    for j in qtl_list:
        # print(f"python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --h2 /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats/{i} --ref-ld-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_baselineLD_v2.2_ldscores/baselineLD.,/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_annot/{j}. --frqfile-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_plinkfiles/1000G.EAS.QC. --w-ld-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_weights_hm3_no_MHC/weights.hm3_noMHC. --overlap-annot --print-cov --print-coefficients --print-delete-vals --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/{gwas_id}_{j}")
        print(f"python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --h2 /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats/{i} --ref-ld-chr /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/EUR/{j}. --frqfile-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/ --w-ld-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_weights_hm3_no_MHC/ --overlap-annot --print-cov --print-coefficients --print-delete-vals --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/{gwas_id}_{j}")

python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --h2 /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats/Cardiovascular_EUR_F902001.sumstats.gz --ref-ld-chr /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/EUR/inosineQTL. --frqfile-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/ --w-ld-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_weights_hm3_no_MHC/ --overlap-annot --print-cov --print-coefficients --print-delete-vals --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/Cardiovascular_EUR_F902001_inosineQTL
python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --h2 /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats/Cardiovascular_EUR_F902001.sumstats.gz --ref-ld-chr /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/l2/EUR/puQTL. --frqfile-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_EUR_Phase3_plink/ --w-ld-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EAS/1000G_Phase3_EAS_weights_

In [ ]:
python /mnt/hpc/home/xuxinran/GWAS/ldsc/ldsc.py --h2 /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/gwas_sumstats/Cardiovascular_EUR_F902001.sumstats.gz --ref-ld-chr /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/qtl_info/EUR/inosineQTL. --frqfile-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/1000G_Phase3_EUR_frq/1000G.EUR.QC. 
--w-ld-chr /mnt/hpc/home/xuxinran/GWAS/ldsc/LDSC_PATH/eur_w_ld_chr --overlap-annot --print-cov --print-coefficients --print-delete-vals --out /mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/ldsc/Cardiovascular_EUR_F902001_inosineQTL
--w-ld-chr /project2/xinhe/m6A/ldsc/LDSCORE/1000G_Phase3_weights_hm3_no_MHC/weights.hm3_noMHC.